# Tokenize pre-mRNA sequences

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
import multiprocessing as mp
import pandas as pd
import random
import os

In [ ]:
os.chdir('/picb/rnasys2/zhouhanwen/github/LAMAR/')

In [ ]:
# tokenizer
tokenizer_path = 'tokenizer/single_nucleotide/'
model_max_length = 1026
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, model_max_length=model_max_length)

In [ ]:
def group_texts(examples):
    tokenized_inputs = tokenizer(examples["seq"], truncation=True, max_length=tokenizer.model_max_length)
    tokenized_inputs["labels"] = examples["label"]
    return tokenized_inputs

In [ ]:
# paths of training set and testing set
train_set_path = 'SpliceSitePred/data/trainset_split_768.json'
# path of saving file
data_path = 'SpliceSitePred/data/ss_single_nucleotide'

In [ ]:
# This step takes ~10 mins. Please wait a moment.
dataset = load_dataset("json", data_files=train_set_path)
data = dataset.map(group_texts, batched=True, num_proc=mp.cpu_count())
data = data.remove_columns(["seq", "label"])
data = data["train"].train_test_split(test_size=0.001, shuffle=True, seed=42)
data.save_to_disk(data_path, num_proc=mp.cpu_count())

In [ ]:
data